In [2]:
# Import Libraries
import osmnx as ox
import pandas as pd

In [4]:
# 1. Pull Restaurants + Breweries
place_name = "Boulder, Colorado, USA"
tags = {"amenity": ["restaurant", "bar", "pub", "brewery"]}

In [5]:
# --- STEP 1: Pull Restaurants, Bars, Pubs, Breweries ---
print("Pulling restaurants/bars/pubs/breweries...")

restaurant_tags = {"amenity": ["restaurant", "bar", "pub", "brewery"]}
places = ox.features_from_place(place_name, tags=restaurant_tags)
print(f"Pulled {len(places)} total places.")

Pulling restaurants/bars/pubs/breweries...
Pulled 193 total places.


In [6]:
# --- STEP 2: Filter Dog-Friendly Restaurants/Breweries ---
# --- STEP 2: Filter Dog-Friendly Restaurants/Breweries ---
def is_dog_friendly(row):
    dog_tags = ['dog', 'dogs', 'dog_friendly', 'outdoor_seating', 'notes']
    for tag in dog_tags:
        val = row.get(tag)
        if isinstance(val, str) and "yes" in val.lower():
            return True
    return False

print("Filtering for dog-friendly restaurants and breweries...")
dog_friendly_places = places[places.apply(is_dog_friendly, axis=1)]
print(f"Found {len(dog_friendly_places)} dog-friendly restaurants/breweries.")


Filtering for dog-friendly restaurants and breweries...
Found 17 dog-friendly restaurants/breweries.


In [7]:
# 3. Reproject to a Projected CRS (meters, not lat/lon)
dog_friendly_places = dog_friendly_places.to_crs(epsg=3857)  # Web Mercator for map work


In [8]:
# 4. Fix Geometry (Centroid)
dog_friendly_places.loc[:, 'geometry'] = dog_friendly_places.geometry.centroid

In [9]:
# 5. Back to WGS84 (lat/lon)
dog_friendly_places = dog_friendly_places.to_crs(epsg=4326)

In [10]:
# 6. Extract Info
dog_friendly_places['latitude'] = dog_friendly_places.geometry.y
dog_friendly_places['longitude'] = dog_friendly_places.geometry.x
dog_friendly_places['place_type'] = dog_friendly_places['amenity']

dog_friendly_df = dog_friendly_places[['name', 'latitude', 'longitude', 'place_type']].dropna()


In [11]:
# 7. Save Dog-Friendly Places
dog_friendly_df.to_csv("../boulder-trails-app/data/dog_friendly_places.csv", index=False)
print(f"Saved {len(dog_friendly_df)} dog-friendly places!")

Saved 17 dog-friendly places!


In [12]:
# 8. Pull Dog Parks
dog_park_tags = {"leisure": "dog_park"}
print("Pulling dog parks...")
dog_parks = ox.features_from_place(place_name, tags=dog_park_tags)
print(f"Pulled {len(dog_parks)} dog parks.")

Pulling dog parks...
Pulled 9 dog parks.


In [13]:

# 9. Reproject dog parks to meters
dog_parks = dog_parks.to_crs(epsg=3857)

# 10. Fix Geometry
dog_parks.loc[:, 'geometry'] = dog_parks.geometry.centroid

# 11. Back to lat/lon
dog_parks = dog_parks.to_crs(epsg=4326)


In [14]:
# 12. Extract Info
dog_parks['latitude'] = dog_parks.geometry.y
dog_parks['longitude'] = dog_parks.geometry.x

dog_parks_df = dog_parks[['name', 'latitude', 'longitude']].dropna()

# 13. Save Dog Parks
dog_parks_df.to_csv("../boulder-trails-app/data/dog_parks.csv", index=False)
print(f"Saved {len(dog_parks_df)} dog parks!")

Saved 6 dog parks!


In [15]:
# pull in Vet info
# --- STEP 1: Pull Veterinary Places ---
vet_tags = {"amenity": "veterinary"}
print("Pulling veterinary clinics and hospitals...")
vets = ox.features_from_place(place_name, tags=vet_tags)
print(f"Pulled {len(vets)} vet places.")

Pulling veterinary clinics and hospitals...
Pulled 7 vet places.


In [16]:
# --- STEP 2: Fix Geometry ---
vets = vets.to_crs(epsg=3857)  # Project to meters
vets.loc[:, 'geometry'] = vets.geometry.centroid
vets = vets.to_crs(epsg=4326)  # Back to lat/lon

In [17]:
# --- STEP 3: Extract Info ---
vets['latitude'] = vets.geometry.y
vets['longitude'] = vets.geometry.x

vets_df = vets[['name', 'latitude', 'longitude']].dropna()

In [21]:
# --- STEP 4: Build Final DataFrame ---
vets_df = vets[['name', 'latitude', 'longitude']].dropna()

In [22]:
# --- STEP 5: Save Combined CSV ---
vets_df.to_csv("../boulder-trails-app/data/veterinarians.csv", index=False)
print(f"Saved {len(vets_df)} veterinarians.")


Saved 7 veterinarians.
